# Ejercicio: Análisis de Series Temporales
Para este ejercicio vas a trabajar con el histórico de valores en bolsa de una empresa que se dedica a vender muebles. Se pide:
1. Importa los datos 'df_furniture.csv'.
2. Trata la fecha como texto. Crea una columna para el año, mes y dia, y a partir de esas columnas crea la de fecha, en formato datetime de pandas.
3. ¿De qué fecha a qué fecha va el histórico?
4. ¿Cómo es la serie temporal? ¿Tiene tendencia? ¿Aprecias alguna estacionalidad?
5. ¿Podrías demostrar de dos maneras estadísticas que la serie no es estacionaria?
6. Parece que hay una tendencia, pero no se aprecia bien con tanto pico en la serie ¿Conoces algún método para representar mejor la tendencia?
7. Estaría bien visualizar todas las componentes de la serie temporal por separado.
8. Demuestra mediante una gráfica de autocorrelación que la serie es estacional.
9. ¿Qué dependencia tiene cada instante de sus ultimos 5 *lags*? Demuéstralo mediante una matriz de correlación. ¿Esto es bueno para los modelos?
10. Ya hemos visto que hay una estacionalidad ¿Cada cuántos instantes se cumple ese patrón?
11. Para un futuro modelado de los datos necesitaremos eliminar la tendencia ¿Cómo hacemos esto?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Cargamos los datos

In [ ]:
df = pd.read_csv('data/df_furniture.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## 2. Tratar la fecha como texto y crear nuevas columnas

In [ ]:
df[df['Month'].isna()]

In [ ]:
df[13:25]

In [ ]:
df['Month'] = pd.to_datetime(df['Month'])
df.info()

## 3. ¿De qué fecha a qué fecha va el histórico?

In [ ]:
df['Month'].min()

In [ ]:
df['Month'].max()

In [ ]:
fechas = pd.date_range(df['Month'].min(), df['Month'].max(), freq="MS")
fechas

In [ ]:
len(fechas)

In [ ]:
df["Month"] = fechas
df.head()

In [ ]:
df.info()

## 4. ¿Cómo es la serie temporal? ¿Tiene tendencia? ¿Aprecias alguna estacionalidad?

In [ ]:
df.index = df['Month']
df.head()
# set_index()   

In [ ]:
df.drop(columns=['Month'], inplace=True)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(df)

Podemos observar que tiene una tendencia positiva, tiene una estacionalidad anual, cada año se está repitiendo un mismo patrón. Todo esto nos hace indicar que la serie temporal no es estacionaria.

## 5. ¿Podrías demostrar de dos maneras estadísticas que la serie no es estacionaria?

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df['furniture_price_adjusted'].values)
result[1]

Como el nivel de significación (p-valor) es mayor a 0.05, nos quedamos con la hipótesis nula de que no es estacionaria, y rechazaremos la hipótesis alternativa de que es estacionaria.

In [ ]:
int(len(df)/2)

In [ ]:
df[:int(len(df)/2)].describe()

In [ ]:
df[int(len(df)/2):].describe()

Comprobamos también que varían los estadísticos con el tiempo, por lo tanto podemos volver a afirmar que esta serie no es estacionaria.

## 6. ¿Conoces algún método para representar mejor la tendencia?

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Additive Decomposition
result_add = seasonal_decompose(df['furniture_price_adjusted'], model = 'additive', extrapolate_trend = 'freq')
result_mult = seasonal_decompose(df['furniture_price_adjusted'], model = 'multiplicative', extrapolate_trend = 'freq')

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(result_add.trend)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(result_mult.trend)

In [ ]:
df_ma = df['furniture_price_adjusted'].rolling(5).mean()
plt.figure(figsize=(15,10))
plt.plot(df_ma)

## 7. Estaría bien visualizar todos las componentes de la serie temporal por separado

In [ ]:
result_add.plot();

In [ ]:
result_mult.plot();

## 8. Demuestra mediante una gráfica de autocorrelación que la serie es estacional

In [ ]:
from pandas.plotting import autocorrelation_plot

plt.figure(figsize=(15,10))
autocorrelation_plot(df['furniture_price_adjusted']);

Podemos observar los picos de autocorrelación, que nos indica su estacionalidad, se repite un patrón cada año, es decir, cada 12 meses.

## 9. ¿Qué dependencia tiene cada instante de sus últimos 5 *lags*? Demuéstralo mediante una matriz de correlación.

In [ ]:
df.corr()

In [ ]:
for i in range(5,0,-1):
    df['t-'+str(i)] = df['furniture_price_adjusted'].shift(i)

df.dropna(inplace=True)
df

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
# plt.figure(figsize=(15,10))
plot_acf(df['furniture_price_adjusted']);

## 10. Ya hemos visto que hay estacionalidad. ¿Cada cuántos instantes se cumple ese patrón?

Hemos comprobado en apartados anteriores que es de 12 meses.

## 11. Para un futuro modelado de los datos necesitaremos eliminar la tendencia. ¿Cómo hacemos esto?

In [ ]:
len(result_add.trend)

In [ ]:
len(df)

In [ ]:
df['detrend_values'] = df['furniture_price_adjusted'] - result_add.trend
df

In [ ]:
plt.plot(df['detrend_values'])

In [ ]:
df['detrend_values_mult'] = df['furniture_price_adjusted'] - result_mult.trend
df

In [ ]:
plt.plot(df['detrend_values_mult'])